# IMPORTS

In [137]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path

from sklearn import linear_model
import statsmodels.api as sm

# SETUP

In [301]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [452]:
# Paths
path_dados_trafego = PROJECT_DIRS["DADOS_TRAFEGO_AJUSTADOS_TRIMESTRAL_DIR"]
path_dados_PIB = PROJECT_DIRS["DADOS_PIB_DIR"]
# path_output = TODO

In [453]:
periodos = [['2010Q1','2024Q1'], ['2015Q4','2024Q1']] # formato Y/M, inclusive

In [454]:
path_outputs = PROJECT_DIRS["DADOS_REGRESSAO_DIR"]

# CARREGANDO p/MEMORIA OS DADOS

In [512]:
df_veqs_comercial = pd.read_parquet(path_dados_trafego / 'df_VEQS_COMERCIAL_trimestral_saz_adjusted.parquet')
df_veqs_passeio = pd.read_parquet(path_dados_trafego / 'df_VEQS_PASSEIO_trimestral_saz_adjusted.parquet')


dict_veqs = {'VEQS_COMERCIAL':df_veqs_comercial,
           'VEQS_PASSEIO':df_veqs_passeio}

In [513]:
dict_veqs['VEQS_COMERCIAL'].head()

,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,...,ECOPONTE,ECOSUL,ECOVIAS DO CERRADO,MSVIA,NOVADUTRA,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA SUL
trimestre,,,,,,,,,,,,,,,,,,,,,
2010Q1,2.026498e+07,5.615449e+06,1.660783e+07,4.757191e+06,2.720145e+07,0.0,3.523528e+06,3.050045e+06,0.0,2.035210e+06,...,0.0,3.326576e+06,0.0,0.0,2.515616e+07,3.215201e+06,3.693265e+06,0.0,0.0,0.0
2010Q2,2.202874e+07,5.878429e+06,1.726165e+07,4.926565e+06,2.872619e+07,0.0,3.484878e+06,3.251909e+06,0.0,2.076498e+06,...,0.0,3.631615e+06,0.0,0.0,3.057997e+07,3.353519e+06,3.855897e+06,0.0,0.0,0.0
2010Q3,2.336369e+07,6.071373e+06,1.781558e+07,4.921330e+06,2.911318e+07,0.0,3.468492e+06,3.297070e+06,0.0,2.168994e+06,...,0.0,3.559840e+06,0.0,0.0,3.038413e+07,3.180379e+06,3.927718e+06,0.0,0.0,0.0
2010Q4,2.655924e+07,6.304882e+06,1.824895e+07,4.944872e+06,2.984123e+07,0.0,3.617589e+06,3.509909e+06,0.0,2.282016e+06,...,0.0,3.887337e+06,0.0,0.0,3.029005e+07,3.141333e+06,4.040486e+06,0.0,0.0,0.0
2011Q1,2.730935e+07,6.313404e+06,1.779003e+07,5.308139e+06,3.039561e+07,0.0,3.625161e+06,3.578344e+06,0.0,2.232230e+06,...,0.0,4.049414e+06,0.0,0.0,3.078823e+07,3.112422e+06,4.053176e+06,0.0,0.0,0.0


In [514]:
# Dados de PIB
df_pib = pd.read_parquet(path_dados_PIB / 'PIB-Ipea-trimestral.parquet')

In [515]:
df_pib.head()

,PIB_real_pct
Data,
1997Q1,3.40
1997Q2,4.75
1997Q3,1.79
1997Q4,3.74
1998Q1,1.01


# FUNCÕES

In [493]:
def remove_outliers(df_raw, col_dados, thresh=2.0):
    df = pd.DataFrame(index=df_raw.index)
    df['Dados_brutos'] = df_raw.dropna()
    df['mean'] = df['Dados_brutos'].mean()
    df['std'] = df['Dados_brutos'].std()
    df['dist_std_mean'] = (df['Dados_brutos'] - df['mean']) / df['std']
    
    df = df.loc[abs(df['dist_std_mean']) < (thresh + 1e-8)]
    df[col_dados] = df['Dados_brutos']
    
    return df[col_dados]

In [294]:
dict_veqs['VEQS_COMERCIAL']

,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,...,ECOPONTE,ECOSUL,ECOVIAS DO CERRADO,MSVIA,NOVADUTRA,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA SUL
trimestre,,,,,,,,,,,,,,,,,,,,,
2010Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010Q2,8.703506,4.683158,3.936810,3.560385,5.605359,NaN,-1.096916,6.618380,NaN,2.028683,...,NaN,9.169760,NaN,NaN,21.560565,4.302003,4.403486,NaN,NaN,NaN
2010Q3,6.060054,3.282237,3.209042,-0.106259,1.347145,NaN,-0.470203,1.388768,NaN,4.454427,...,NaN,-1.976396,NaN,NaN,-0.640430,-5.162942,1.862616,NaN,NaN,NaN
2010Q4,13.677392,3.846073,2.432502,0.478361,2.500782,NaN,4.298602,6.455387,NaN,5.210830,...,NaN,9.199780,NaN,NaN,-0.309612,-1.227714,2.871100,NaN,NaN,NaN
2011Q1,2.824283,0.135156,-2.514727,7.346330,1.857749,NaN,0.209325,1.949766,NaN,-2.181686,...,NaN,4.169337,NaN,NaN,1.644673,-0.920334,0.314063,NaN,NaN,NaN
2011Q2,0.843995,4.542800,4.358144,-4.477948,1.380957,NaN,1.930188,1.969721,NaN,4.769072,...,NaN,-0.797074,NaN,NaN,2.534610,5.391294,3.728836,NaN,NaN,NaN
2011Q3,2.815287,2.506417,-0.434988,-2.756268,-0.476911,NaN,-0.113539,2.528723,NaN,1.062157,...,NaN,0.514010,NaN,NaN,1.717296,-1.463423,0.893941,NaN,NaN,NaN
2011Q4,-0.517499,-1.102012,4.675099,2.685460,1.138740,NaN,2.678625,2.524919,NaN,-1.581064,...,NaN,1.040485,NaN,NaN,-1.823058,-8.036574,1.975932,NaN,NaN,NaN
2012Q1,3.686592,7.575134,-1.273625,0.198446,0.671387,NaN,3.391624,2.378053,NaN,7.248533,...,NaN,-5.230033,NaN,NaN,-0.962275,-22.315271,3.092242,NaN,NaN,NaN


In [ ]:
def regressao(df_y, df_periodos, conc, df_X, periodo, fit_intercept=True):
    y_train = df_y[conc]
    
    y_train = calc_perc_change(pd.DataFrame(y_train))
    X_train = filter_PIB_periodo(df_PIB, periodo)
    
    y_train = pd.DataFrame(y_train)
    X_train = pd.DataFrame(X_train)
    
    X_train = X_train.loc[y_train.index,:] # O PIB vai ter um mês a mais (o mês inicial)

    if fit_intercept:
        X_train = sm.add_constant(X_train)
    
    model = sm.OLS(y_train, X_train).fit()
    
    return model

In [29]:
# OLD
# def regressao(df_y, df_periodos, conc, df_X, periodo, fit_intercept=True):
#     y_train = df_y[conc]
    
#     y_train = calc_perc_change(pd.DataFrame(y_train))
#     X_train = filter_PIB_periodo(df_PIB, periodo)
    
#     y_train = pd.DataFrame(y_train)
#     X_train = pd.DataFrame(X_train)
    
#     X_train = X_train.loc[y_train.index,:] # O PIB vai ter um mês a mais (o mês inicial)

#     if fit_intercept:
#         X_train = sm.add_constant(X_train)
    
#     model = sm.OLS(y_train, X_train).fit()
    
#     return model

In [ ]:
# OLD
# def regressao(df_y, df_periodos, conc, df_X, periodo, fit_intercept=True):
#     y_train = df_y[conc]
    
#     y_train = calc_perc_change(pd.DataFrame(y_train))
#     X_train = filter_PIB_periodo(df_PIB, periodo)
    
#     y_train = pd.DataFrame(y_train)
#     X_train = pd.DataFrame(X_train)
    
#     X_train = X_train.loc[y_train.index,:] # O PIB vai ter um mês a mais (o mês inicial)

#     if fit_intercept:
#         X_train = sm.add_constant(X_train)
    
#     model = sm.OLS(y_train, X_train).fit()
    
#     return model


In [69]:
def make_models(df_veqs, df_periodos, df_X, periodo, fit_intercept=True):
    concs = filter_trafego_periodo(df_veqs, df_periodos, periodo).columns
    df_stats = pd.DataFrame(index=concs)
    
    for conc in concs:
        model = regressao(df_veqs, df_periodos, conc, df_X, periodo, fit_intercept=fit_intercept)
        param_name = model.params.index
        df_stats.loc[conc, 'R2'] = model.rsquared
        df_stats.loc[conc, f'{param_name[0]}-coef'] = model.params.iloc[0]
        df_stats.loc[conc, f'{model.pvalues.index[0]}-p-valor'] = model.pvalues.iloc[0]
        df_stats.loc[conc, 'n'] = len(filter_trafego_periodo(df_veqs, df_periodos, periodo).iloc[1:,:])

    return df_stats

# Tratando os dados

## PIB

In [516]:
# filtrando PIB para o periodo
df_pib = df_pib.loc[periodos[0][0] : periodos[0][1]]

In [517]:
len(df_pib)

57

In [518]:
# Removendo outliers do PIB
df_pib = remove_outliers(df_pib, 'PIB_real_pct', thresh=2.0)

In [519]:
len(df_pib)

54

## Trafego

In [520]:
# trocando zeros para NaN
for veq, df in dict_veqs.items():
    for col in df:
        df[col] = df[col].apply(lambda x: x if (abs(x) > 1e-8) else None)

In [521]:
# Convertendo os dados para variação relativa em pct (=PIB)
for veq, df in dict_veqs.items():
    for col in df:
        df[col] = df[col].dropna().pct_change().replace([np.inf, -np.inf], None) * 100

In [522]:
# retirando outliers
for veq, df in dict_veqs.items():
    for col in df:
        len1 = len(df[col].dropna())
        df[col] = remove_outliers(df[col], col, thresh=2.0)
        len2 = len(df[col].dropna())
        print(len2-len1, veq, col) 
        # OBS: outliers ficam como NaN

-5 VEQS_COMERCIAL AUTOPISTA FERNÃO DIAS
-2 VEQS_COMERCIAL AUTOPISTA FLUMINENSE
-4 VEQS_COMERCIAL AUTOPISTA LITORAL SUL
-3 VEQS_COMERCIAL AUTOPISTA PLANALTO SUL
-2 VEQS_COMERCIAL AUTOPISTA REGIS BITTENCOURT
-1 VEQS_COMERCIAL CONCEBRA
-1 VEQS_COMERCIAL CONCEPA
-3 VEQS_COMERCIAL CONCER
-3 VEQS_COMERCIAL CRO
-2 VEQS_COMERCIAL CRT
-3 VEQS_COMERCIAL ECO050
-2 VEQS_COMERCIAL ECO101 CONCESSIONARIA DE RODOVIAS S/A
-2 VEQS_COMERCIAL ECOPONTE
-4 VEQS_COMERCIAL ECOSUL
0 VEQS_COMERCIAL ECOVIAS DO CERRADO
-1 VEQS_COMERCIAL MSVIA
-3 VEQS_COMERCIAL NOVADUTRA
-4 VEQS_COMERCIAL RODOVIA DO AÇO
-1 VEQS_COMERCIAL TRANSBRASILIANA
-3 VEQS_COMERCIAL VIA 040
-2 VEQS_COMERCIAL VIA BAHIA
-1 VEQS_COMERCIAL VIA SUL
-3 VEQS_PASSEIO AUTOPISTA FERNÃO DIAS
-3 VEQS_PASSEIO AUTOPISTA FLUMINENSE
-5 VEQS_PASSEIO AUTOPISTA LITORAL SUL
-3 VEQS_PASSEIO AUTOPISTA PLANALTO SUL
-4 VEQS_PASSEIO AUTOPISTA REGIS BITTENCOURT
-3 VEQS_PASSEIO CONCEBRA
-2 VEQS_PASSEIO CONCEPA
-2 VEQS_PASSEIO CONCER
-3 VEQS_PASSEIO CRO
-3 VEQS_PASSEIO 

# REGRESSÃO

## Periodo 1

In [70]:
model_11_23 = make_models(df_veqs_comercial, df_periodos, df_PIB, ['2010-01','2022-01'], fit_intercept=False)
model_11_23

,R2,PIB_mensal_real_%-coef,PIB_mensal_real_%-p-valor,n
concessionaria,,,,
NOVADUTRA,0.067417,0.478604,0.001612,144.0
TRANSBRASILIANA,0.033456,0.266550,0.027659,144.0
RODOVIA DO AÇO,0.090588,0.708618,0.000235,144.0
AUTOPISTA FERNÃO DIAS,0.071743,0.525864,0.001125,144.0
AUTOPISTA FLUMINENSE,0.046036,0.371503,0.009555,144.0
CRT,0.121483,0.578572,0.000017,144.0
CONCER,0.063499,0.435040,0.002231,144.0
AUTOPISTA REGIS BITTENCOURT,0.050485,0.431861,0.006587,144.0
AUTOPISTA PLANALTO SUL,0.073894,0.542131,0.000941,144.0


In [71]:
model_11_23['R2'].mean()

np.float64(0.0631262914110482)

In [72]:
model_11_23['PIB_mensal_real_%-p-valor'].mean()

np.float64(0.008393286981317997)

In [73]:
model_11_23['PIB_mensal_real_%-coef'].mean()

np.float64(0.5019885481245611)

## Periodo 2017-2023

In [74]:
model_17_23 = make_models(df_veqs_comercial, df_periodos, df_PIB, ['2015-09','2022-01'], fit_intercept=False)
model_17_23

,R2,PIB_mensal_real_%-coef,PIB_mensal_real_%-p-valor,n
concessionaria,,,,
MSVIA,0.039159,0.697116,0.084500,76.0
CRO,0.021176,0.427447,0.206667,76.0
VIA 040,0.060703,0.513133,0.030771,76.0
ECOPONTE,0.001938,0.302719,0.703825,76.0
CONCEBRA,0.061970,0.518853,0.029022,76.0
ECO050,0.026910,1.015849,0.153981,76.0
ECO101 CONCESSIONARIA DE RODOVIAS S/A,0.038257,0.390932,0.088240,76.0
VIA BAHIA,0.078840,0.668692,0.013378,76.0
NOVADUTRA,0.105666,0.762540,0.003920,76.0


In [75]:
model_17_23['R2'].mean()

np.float64(0.05627839177528739)

In [78]:
model_17_23['PIB_mensal_real_%-p-valor'].mean()

np.float64(0.09370676845288983)

In [80]:
model_17_23['PIB_mensal_real_%-coef'].mean()

np.float64(0.5969748377139612)

# Salvando para excel

In [82]:
with pd.ExcelWriter(path_outputs / 'resultado_modelos_PIB_mensal.xlsx') as writer:
    # df_resultados = pd.DataFrame(index=df_periodos.index)
    for p in periodos:
        for key, veq in dict_veqs.items():
            df_resultados = make_models(veq, df_periodos, df_PIB, p, fit_intercept=False)
            sheet_name = f'{p[0]}-{p[1]}-{key}_mensal'
            df_resultados.to_excel(writer, sheet_name = sheet_name)        

/home/andcm/.cache/pypoetry/virtualenvs/infra-data-J9p6eQia-py3.9/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/andcm/.cache/pypoetry/virtualenvs/infra-data-J9p6eQia-py3.9/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/andcm/.cache/pypoetry/virtualenvs/infra-data-J9p6eQia-py3.9/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/andcm/.cache/